In [5]:

import numpy as np
import os, glob, sys
import pandas as pd


iter = 1
print('Proecssing iteration', iter)
# print('basin_on_one_node', basin_on_one_node)

inpath = '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO_bigrange'
outpath = '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO_bigrange/run_model_mpiserial_emutest'
os.makedirs(outpath, exist_ok=True)

# basins = [f'level1_{i}' for i in range(1, 627, 10)] 
basins = [f'level1_{i}' for i in range(1, 627) if not i in np.arange(1, 627, 10)] 
bnum = len(basins)

cpu_per_node = 128

jobparams = [ "#PBS -N MOAcalib", "#PBS -q main", 
              "#PBS -l select=1:ncpus=128", "#PBS -l walltime=12:00:00", 
              "#PBS -l job_priority=economy",
              "#PBS -A P08010000",
             "\n",
              "module load conda nco cdo",
             "conda activate npl-2024a-tgq",
             "\n",
            ]



allcommands = []
for j in range(len(basins)):
    infileij = f'{inpath}/{basins[j]}_MOASMOcalib/run_model/iter0/commands_run_iter0.txt'
    with open(infileij, 'r') as f:
        linesj = f.readlines()
    for i in range(100): # use the first 100 lines as a template
        l = linesj[i]

        # for new parameter sets
        l = l.replace('iter0', f'iter{iter}')
        l = l.replace('ctsm_outputs', 'ctsm_outputs_emutest')
        l = l.replace('param_sets', 'param_sets_emutest')

        l = l.strip().split(' ')
        
        allcommands.append(' '.join(l[:-1]))

allcommands = np.array(allcommands)

trials_on_one_node = cpu_per_node * 5
nbatch = int(len(allcommands)/trials_on_one_node) + 1


for i in range(nbatch):
    bstart = i*trials_on_one_node
    bend = (i+1)*trials_on_one_node
    if bend>len(allcommands):
        bend = len(allcommands)

    outpathi = f'{outpath}/batch{i}'
    os.makedirs(outpathi, exist_ok=True)
    
    # generate command line file
    newcommands = allcommands[bstart:bend]


    outfile_newcom = f'{outpathi}/batch_{i}.txt'
    with open(outfile_newcom, 'w') as f:
        for l in newcommands:
            _ = f.write(l+'\n')

    # generate submission list
    filesub = f'{outpathi}/submission.sh'
    with open(filesub, 'w') as f:
        for l in jobparams:
            _ = f.write(l+'\n')
        command = f"parallel -j {cpu_per_node} < {outfile_newcom}"
        _ = f.write(command+'\n')



Proecssing iteration 1
